In [ ]:
# Sparsely-Gated Mixture-of-Experts Layers.
# See "Outrageously Large Neural Networks"
# https://arxiv.org/abs/1701.06538
#
# Author: David Rau
#
# The code is based on the TensorFlow implementation:
# https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/utils/expert_utils.py

import torch
import torch.nn as nn
from torch.distributions.normal import Normal
from mlp import MLP
import numpy as np


class SparseDispatcher(object):
    """Helper for implementing a mixture of experts.
    The purpose of this class is to create input minibatches for the
    experts and to combine the results of the experts to form a unified
    output tensor.
    There are two functions:
    dispatch - take an input Tensor and create input Tensors for each expert. 将input送到各个expert
    combine - take output Tensors from each expert and form a combined output 将expert的各个输出根据gate权值合并成一个输出
      Tensor.  Outputs from different experts for the same batch element are
      summed together, weighted by the provided "gates".
    The class is initialized with a "gates" Tensor, which specifies which
    batch elements go to which experts, and the weights to use when combining
    the outputs.  Batch element b is sent to expert e iff gates[b, e] != 0.
    The inputs and outputs are all two-dimensional [batch, depth].
    Caller is responsible for collapsing additional dimensions prior to
    calling this class and reshaping the output to the original shape.
    See common_layers.reshape_like().
    Example use:
    gates: a float32 `Tensor` with shape `[batch_size, num_experts]`
    inputs: a float32 `Tensor` with shape `[batch_size, input_size]`
    experts: a list of length `num_experts` containing sub-networks.
    dispatcher = SparseDispatcher(num_experts, gates)
    expert_inputs = dispatcher.dispatch(inputs)
    expert_outputs = [experts[i](expert_inputs[i]) for i in range(num_experts)]
    outputs = dispatcher.combine(expert_outputs)
    The preceding code sets the output for a particular example b to:
    output[b] = Sum_i(gates[b, i] * experts[i](inputs[b]))
    This class takes advantage of sparsity in the gate matrix by including in the
    `Tensor`s for expert i only the batch elements for which `gates[b, i] > 0`.
    """
    def __init__(self, num_experts, gates):
        """Create a SparseDispatcher."""

        self._gates = gates
        self._num_experts = num_experts
        # sort experts
        # dim=0是batch，没用；dim=1是expert坐标，有用
        sorted_experts, index_sorted_experts = torch.nonzero(gates).sort(0)
        # drop indices
        # _expert_index就是这个batch用了哪些expert
        _, self._expert_index = sorted_experts.split(1, dim=1)
        # get according batch index for each expert
        # _batch_index得到expert都是哪些batch的
        self._batch_index = sorted_experts[index_sorted_experts[:, 1], 0]
        # calculate num samples that each expert gets
        # 每个experts被选了多少次
        self._part_sizes = list((gates > 0).sum(0).numpy())
        # expand gates to match with self._batch_index，方便下面取权值
        gates_exp = gates[self._batch_index.flatten()]  # [选用的experts，num_experts]
        # 获得每个experts的权重
        self._nonzero_gates = torch.gather(gates_exp, 1, self._expert_index)

    def dispatch(self, inp):
        """Create one input Tensor for each expert.
        The `Tensor` for a expert `i` contains the slices of `inp` corresponding
        to the batch elements `b` where `gates[b, i] > 0`.
        Args:
          inp: a `Tensor` of shape "[batch_size, <extra_input_dims>]`
        Returns:
          a list of `num_experts` `Tensor`s with shapes
            `[expert_batch_size_i, <extra_input_dims>]`.
        """

        # assigns samples to experts whose gate is nonzero

        # expand according to batch index so we can just split by _part_sizes
        # inp_exp
        inp_exp = inp[self._batch_index].squeeze(1)  # [选用的experts=k*batch,input_dim]
        # 使用的expert返回对应batch的输入，不使用的返回0
        # 因为每个batch选择的experts是固定的，所以_part_sizes.sum()=k*batch
        return torch.split(inp_exp, self._part_sizes, dim=0)  # [num_experts, input_dim]

    def combine(self, expert_out, multiply_by_gates=True):
        """Sum together the expert output, weighted by the gates.
        The slice corresponding to a particular batch element `b` is computed
        as the sum over all experts `i` of the expert output, weighted by the
        corresponding gate values.  If `multiply_by_gates` is set to False, the
        gate values are ignored.
        Args:
          expert_out: a list of `num_experts` `Tensor`s, each with shape
            `[expert_batch_size_i, <extra_output_dims>]`.
          multiply_by_gates: a boolean
        Returns:
          a `Tensor` with shape `[batch_size, <extra_output_dims>]`.
        """
        # apply exp to expert outputs, so we are not longer in log space
        # 因为expert最后一层的激活函数是logsoftmax
        stitched = torch.cat(expert_out, 0).exp()

        if multiply_by_gates:
            stitched = stitched.mul(self._nonzero_gates)
        zeros = torch.zeros(self._gates.size(0), expert_out[-1].size(1), requires_grad=True) 
        # combine samples that have been processed by the same k experts
        combined = zeros.index_add(0, self._batch_index, stitched.float())
        # add eps to all zero values in order to avoid nans when going back to log space
        # 将所有为0的位置设置为eps
        combined[combined == 0] = np.finfo(float).eps
        # back to log space
        return combined.log()

    def expert_to_gates(self):
        """Gate values corresponding to the examples in the per-expert `Tensor`s.
        Returns:
          a list of `num_experts` one-dimensional `Tensor`s with type `tf.float32`
              and shapes `[expert_batch_size_i]`
        """
        # split nonzero gates for each expert
        return torch.split(self._nonzero_gates, self._part_sizes, dim=0)


class MoE(nn.Module):
    """Call a Sparsely gated mixture of experts layer with 1-layer Feed-Forward networks as experts.
    Args:
    input_size: integer - size of the input
    output_size: integer - size of the input
    num_experts: an integer - number of experts
    hidden_size: an integer - hidden size of the experts
    noisy_gating: a boolean
    k: an integer - how many experts to use for each batch element
    """
    def __init__(self, input_size, output_size, num_experts, hidden_size, noisy_gating=True, k=4):
        super(MoE, self).__init__()
        self.noisy_gating = noisy_gating
        self.num_experts = num_experts
        self.output_size = output_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.k = k
        # instantiate experts
        # 就是普通的MLP
        self.experts = nn.ModuleList([MLP(self.input_size, self.output_size, self.hidden_size) for i in range(self.num_experts)])
        self.w_gate = nn.Parameter(torch.zeros(input_size, num_experts), requires_grad=True)
        self.w_noise = nn.Parameter(torch.zeros(input_size, num_experts), requires_grad=True)

        # 激活函数，relu的平滑版本
        self.softplus = nn.Softplus()
        # 在dim=1维做softmax
        self.softmax = nn.Softmax(1)
        # 从均值为0，方差为1的离散正态分布随机采样
        self.normal = Normal(torch.tensor([0.0]), torch.tensor([1.0]))

        assert (self.k <= self.num_experts)

    def cv_squared(self, x):
        """The squared coefficient of variation of a sample.
        Useful as a loss to encourage a positive distribution to be more uniform.
        Epsilons added for numerical stability.
        Returns 0 for an empty Tensor.
        Args:
        x: a `Tensor`.
        Returns:
        a `Scalar`.
        """
        eps = 1e-10
        # if only num_experts = 1
        if x.shape[0] == 1:
            return torch.Tensor([0])
        return x.float().var() / (x.float().mean()**2 + eps)

    def _gates_to_load(self, gates):
        """Compute the true load per expert, given the gates.
        The load is the number of examples for which the corresponding gate is >0.
        Args:
        gates: a `Tensor` of shape [batch_size, n]
        Returns:
        a float32 `Tensor` of shape [n]
        """
        return (gates > 0).sum(0)

    # 计算所有expert的prob
    def _prob_in_top_k(self, clean_values, noisy_values, noise_stddev, noisy_top_values):
        """Helper function to NoisyTopKGating.
        Computes the probability that value is in top k, given different random noise.
        This gives us a way of backpropagating from a loss that balances the number
        of times each expert is in the top k experts per example.
        In the case of no noise, pass in None for noise_stddev, and the result will
        not be differentiable.
        Args:
        clean_values: a `Tensor` of shape [batch, n].
        noisy_values: a `Tensor` of shape [batch, n].  Equal to clean values plus
          normally distributed noise with standard deviation noise_stddev.
        noise_stddev: a `Tensor` of shape [batch, n], or None
        noisy_top_values: a `Tensor` of shape [batch, m].
           "values" Output of tf.top_k(noisy_top_values, m).  m >= k+1
        Returns:
        a `Tensor` of shape [batch, n].
        """

        batch = clean_values.size(0)
        m = noisy_top_values.size(1)
        # 长度是batch*m
        top_values_flat = noisy_top_values.flatten()
        # *和+都是对值的操作，不是对size的，这是list不是string
        # threshold_position_if_in[i]代表batch_i的权值k+1大的expert在top_values_flat的什么位置
        threshold_positions_if_in = torch.arange(batch) * m + self.k
        # 每个batch权值第k+1大expert的权值
        threshold_if_in = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_in), 1)
        # 与k+1权值比大小，每个batch权值前k大的experts True，其他False
        is_in = torch.gt(noisy_values, threshold_if_in)
        # 每个batch权值第k大的位置
        threshold_positions_if_out = threshold_positions_if_in - 1
        # 每个batch第k大的权值
        threshold_if_out = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_out), 1)
        # is each value currently in the top k.
        # cdf概率分布函数
        prob_if_in = self.normal.cdf((clean_values - threshold_if_in) / noise_stddev)
        prob_if_out = self.normal.cdf((clean_values - threshold_if_out) / noise_stddev)
        prob = torch.where(is_in, prob_if_in, prob_if_out)
        return prob

    # 只有在train的时候才加噪声，类似于dropout
    # 计算所有experts的权值和概率
    def noisy_top_k_gating(self, x, train, noise_epsilon=1e-2):
        """Noisy top-k gating.
          See paper: https://arxiv.org/abs/1701.06538.
          Args:
            x: input Tensor with shape [batch_size, input_size]
            train: a boolean - we only add noise at training time.
            noise_epsilon: a float
          Returns:
            gates: a Tensor with shape [batch_size, num_experts]
            load: a Tensor with shape [num_experts]
        """
        # @ 就是矩阵相乘 ，logit就是权值
        clean_logits = x @ self.w_gate 
        if self.noisy_gating:
            raw_noise_stddev = x @ self.w_noise
            # softplus激活函数，*train是说当不train的时候，不加noise
            noise_stddev = ((self.softplus(raw_noise_stddev) + noise_epsilon) * train)
            # 计算噪声，stddev只是随机数的权值
            noisy_logits = clean_logits + (torch.randn_like(clean_logits) * noise_stddev)
            logits = noisy_logits 
        else:
            logits = clean_logits

        # calculate topk + 1 that will be needed for the noisy gates
        # k是每次用几个，num_experts是一共有几个
        top_logits, top_indices = logits.topk(min(self.k + 1, self.num_experts), dim=1)
        top_k_logits = top_logits[:, :self.k]
        top_k_indices = top_indices[:, :self.k]
        top_k_gates = self.softmax(top_k_logits)

        zeros = torch.zeros_like(logits, requires_grad=True)
        # 只有对应experts有权值，其他experts对应位置权值为0
        # 因为topk返回的是k个值和k个坐标，用这k个值构建gates矩阵，就要按坐标填回去，所以用scatter
        # 所以scatter与topk应该可以说配套使用了，在哪一维用的topk，scatter的时候就设置这一维
        gates = zeros.scatter(1, top_k_indices, top_k_gates)

        if self.noisy_gating and self.k < self.num_experts:
            # clean logits是 *没加噪声的* *所有* experts权值
            # noisy logits是加噪声的所有experts的权值
            # noisy stddev是噪声的权值
            # top logit是前 *k+1* 个experts的权值
            load = (self._prob_in_top_k(clean_logits, noisy_logits, noise_stddev, top_logits)).sum(0)
        else:
            load = self._gates_to_load(gates)
        return gates, load

    def forward(self, x, train=True, loss_coef=1e-2):
        """Args:
        x: tensor shape [batch_size, input_size]
        train: a boolean scalar.
        loss_coef: a scalar - multiplier on load-balancing losses

        Returns:
        y: a tensor with shape [batch_size, output_size].
        extra_training_loss: a scalar.  This should be added into the overall
        training loss of the model.  The backpropagation of this loss
        encourages all experts to be approximately equally used across a batch.
        """
        gates, load = self.noisy_top_k_gating(x, train)
        # calculate importance loss
        importance = gates.sum(0)
        # 计算L_importance
        loss = self.cv_squared(importance) + self.cv_squared(load)
        loss *= loss_coef

        dispatcher = SparseDispatcher(self.num_experts, gates)
        expert_inputs = dispatcher.dispatch(x)
        gates = dispatcher.expert_to_gates()
        expert_outputs = [self.experts[i](expert_inputs[i]) for i in range(self.num_experts)]
        y = dispatcher.combine(expert_outputs)
        return y, loss


In [ ]:

_gates = gates
_num_experts = num_experts
# sort experts
sorted_experts, index_sorted_experts = torch.nonzero(gates).sort(0)
# drop indices
_, self._expert_index = sorted_experts.split(1, dim=1)
# get according batch index for each expert
self._batch_index = sorted_experts[index_sorted_experts[:, 1],0]
# calculate num samples that each expert gets
self._part_sizes = list((gates > 0).sum(0).numpy())
# expand gates to match with self._batch_index
gates_exp = gates[self._batch_index.flatten()]
self._nonzero_gates = torch.gather(gates_exp, 1, self._expert_index)

1. 噪声为什么生成权值*随机数

In [1]:
import torch 
a=torch.arange(4)*3+2
print(a)

tensor([ 2,  5,  8, 11])


In [2]:
from torch.nn import Softplus
x=torch.randn(2,4)
w_gate=torch.randn(4,6)
w_noise=torch.randn(4,6)
noise_epsilon=1e-9
train = True
noisy_gating = True
print("x=",x)
print("w_gate=",w_gate)
clean_logits = x @ w_gate
print("clean_logits=",clean_logits)
softplus=Softplus()
if noisy_gating and train:
    print("w_noise=",w_noise)
    raw_noise_stddev = x @ w_noise
    print("raw_noise_stddev=",raw_noise_stddev)
    noise_stddev = (softplus(raw_noise_stddev) + noise_epsilon)
    print("noise_stddev=",noise_stddev)
    noisy_logits = clean_logits + (torch.randn_like(clean_logits) * noise_stddev)
    print("noisy_logits=",noisy_logits)
    logits = noisy_logits
else:
    logits = clean_logits
print("logits=",logits)

x= tensor([[-0.1236, -0.1572,  0.0284, -1.8244],
        [ 0.2689, -0.0553, -2.3533, -0.2592]])
w_gate= tensor([[-2.1222,  1.3613,  0.2003, -0.2632, -0.3596,  0.8087],
        [-0.1544,  0.7297, -0.1904,  1.1954,  0.2598,  1.8146],
        [-0.1307,  0.5797,  0.5661, -0.1442, -1.2058,  0.0387],
        [-0.5390, -0.8894,  0.6720, -1.5502, -1.3553,  0.3350]])
clean_logits= tensor([[ 1.2663,  1.3560, -1.2047,  2.6686,  2.4420, -0.9954],
        [-0.1148, -0.8081, -1.4420,  0.6042,  3.0779, -0.0607]])
w_noise= tensor([[ 0.1593, -0.3788, -0.3022,  0.2430, -0.8052, -0.4005],
        [ 1.4051, -1.0963,  1.7859,  0.0591,  0.4802, -2.7880],
        [ 0.0601,  0.4878,  0.2107, -1.2015, -1.4441, -2.4711],
        [ 0.1394, -0.2905,  0.0859, -2.0852, -0.3247,  1.2489]])
raw_noise_stddev= tensor([[-0.4932,  0.7629, -0.3942,  3.7308,  0.5754, -1.8608],
        [-0.2125, -1.1139, -0.6982,  3.4299,  3.2395,  5.5379]])
noise_stddev= tensor([[0.4767, 1.1457, 0.5154, 3.7545, 1.0217, 0.1446],
        [0.

In [6]:
from torch.nn import Softmax
softmax=Softmax(1)
k=2 
num_experts = 6
top_logits, top_indices = logits.topk(min(k + 1, num_experts), dim=1)
print("top_logits=",top_logits)
print("top_indices=",top_indices)
top_k_logits = top_logits[:, :k]
top_k_indices = top_indices[:, :k]
top_k_gates = softmax(top_k_logits)
print("top_k_gates=",top_k_gates)
zeros = torch.zeros_like(logits, requires_grad=True)
# 只有对应experts有权值，其他experts对应位置权值为0
gates = zeros.scatter(1, top_k_indices, top_k_gates)
print("gates=",gates)
# if self.noisy_gating and self.k < self.num_experts:
#     load = (self._prob_in_top_k(clean_logits, noisy_logits, noise_stddev, top_logits)).sum(0)


top_logits= tensor([[2.0613, 1.3576, 0.7683],
        [5.0294, 1.8538, 0.1414]])
top_indices= tensor([[4, 0, 1],
        [5, 3, 4]])
top_k_gates= tensor([[0.6690, 0.3310],
        [0.9599, 0.0401]])
gates= tensor([[0.3310, 0.0000, 0.0000, 0.0000, 0.6690, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0401, 0.0000, 0.9599]],
       grad_fn=<ScatterBackward0>)


In [16]:
from torch.distributions.normal import Normal
clean_values=clean_logits
noisy_values=noisy_logits
noisy_top_values=top_logits
batch = clean_values.size(0)
m = noisy_top_values.size(1)
# 长度是batch*m
top_values_flat = noisy_top_values.flatten()
print("noisy_top_values=",noisy_top_values)
print("top_values_flat=",top_values_flat)
# *和+都是对值的操作，不是对size的，这是list不是string
# threshold_position_if_in[i]代表batch_i的最后一个experts在top_values_flat的什么位置
batch=2
threshold_positions_if_in = torch.arange(batch) * m + k
print("threshold_positions_if_in=",threshold_positions_if_in)
threshold_if_in = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_in), 1)
print("threshold_if_in=",threshold_if_in)
is_in = torch.gt(noisy_values, threshold_if_in)
print("is_in=",is_in)
threshold_positions_if_out = threshold_positions_if_in - 1
print("threshold_positions_if_out=",threshold_positions_if_out)
threshold_if_out = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_out), 1)
print("threshold_if_out=",threshold_if_out)
# is each value currently in the top k.
normal=Normal(torch.tensor([0.0]),torch.tensor([1.0]))
print("normal=",normal)
prob_if_in = normal.cdf((clean_values - threshold_if_in) / noise_stddev)
print("prob_if_i=",prob_if_in)
prob_if_out = normal.cdf((clean_values - threshold_if_out) / noise_stddev)
print("prob_if_out",prob_if_out)
prob = torch.where(is_in, prob_if_in, prob_if_out)
print("prob=",prob)

noisy_top_values= tensor([[2.0613, 1.3576, 0.7683],
        [5.0294, 1.8538, 0.1414]])
top_values_flat= tensor([2.0613, 1.3576, 0.7683, 5.0294, 1.8538, 0.1414])
threshold_positions_if_in= tensor([2, 5])
threshold_if_in= tensor([[0.7683],
        [0.1414]])
is_in= tensor([[ True, False, False, False,  True, False],
        [False, False, False,  True, False,  True]])
threshold_positions_if_out= tensor([1, 4])
threshold_if_out= tensor([[1.3576],
        [1.8538]])
normal= Normal(loc: tensor([0.]), scale: tensor([1.]))
prob_if_i= tensor([[8.5193e-01, 6.9601e-01, 6.4492e-05, 6.9362e-01, 9.4931e-01, 0.0000e+00],
        [3.3274e-01, 4.1178e-04, 4.4048e-05, 5.5318e-01, 8.1483e-01, 4.8546e-01]])
prob_if_out tensor([[4.2400e-01, 4.9941e-01, 3.2783e-07, 6.3652e-01, 8.5573e-01, 0.0000e+00],
        [4.4629e-04, 0.0000e+00, 0.0000e+00, 3.5906e-01, 6.4559e-01, 3.6488e-01]])
prob= tensor([[8.5193e-01, 4.9941e-01, 3.2783e-07, 6.3652e-01, 9.4931e-01, 0.0000e+00],
        [4.4629e-04, 0.0000e+00, 0.00

In [18]:
def cv_squared(x):
    eps = 1e-10
    # if only num_experts = 1
    if x.shape[0] == 1:
        return torch.Tensor([0])
    return x.float().var() / (x.float().mean()**2 + eps)
load=prob
print("gates=",gates)
importance=gates.sum(0)
print("importance=",importance)
loss = cv_squared(importance) + cv_squared(load)
print("cv_squared(imp)=",cv_squared(importance))
print("cv_squared(load)=",cv_squared(load))
print("loss=",loss)

gates= tensor([[0.3310, 0.0000, 0.0000, 0.0000, 0.6690, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0401, 0.0000, 0.9599]],
       grad_fn=<ScatterBackward0>)
importance= tensor([0.3310, 0.0000, 0.0000, 0.0401, 0.6690, 0.9599],
       grad_fn=<SumBackward1>)
cv_squared(imp)= tensor(1.4643, grad_fn=<DivBackward0>)
cv_squared(load)= tensor(0.8947)
loss= tensor(2.3589, grad_fn=<AddBackward0>)


In [23]:
print("gates.nonzero=",torch.nonzero(gates))
sorted_experts, index_sorted_experts = torch.nonzero(gates).sort(0)
print("sorted_experts=",sorted_experts)
print("index_sorted_expert=",index_sorted_experts)
# drop indices
_, expert_index = sorted_experts.split(1, dim=1)
print("_=",_)
print("expert_index=",expert_index)
# get according batch index for each expert
print("index_sorted_experts[:,1]=",index_sorted_experts[:,1])
_batch_index = sorted_experts[index_sorted_experts[:, 1], 0]
print("_batch_index=",_batch_index)
# calculate num samples that each expert gets
_part_sizes = list((gates > 0).sum(0).numpy())
print("_part_sizes=",_part_sizes)
# expand gates to match with self._batch_index
gates_exp = gates[_batch_index.flatten()]
print("gates_exp=",gates_exp)
_nonzero_gates = torch.gather(gates_exp, 1, expert_index)
print("_nonzero_gates=",_nonzero_gates)

gates.nonzero= tensor([[0, 0],
        [0, 4],
        [1, 3],
        [1, 5]])
sorted_experts= tensor([[0, 0],
        [0, 3],
        [1, 4],
        [1, 5]])
index_sorted_expert= tensor([[0, 0],
        [1, 2],
        [2, 1],
        [3, 3]])
_= tensor([[0],
        [0],
        [1],
        [1]])
expert_index= tensor([[0],
        [3],
        [4],
        [5]])
index_sorted_experts[:,1]= tensor([0, 2, 1, 3])
_batch_index= tensor([0, 1, 0, 1])
_part_sizes= [1, 0, 0, 1, 1, 1]
gates_exp= tensor([[0.3310, 0.0000, 0.0000, 0.0000, 0.6690, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0401, 0.0000, 0.9599],
        [0.3310, 0.0000, 0.0000, 0.0000, 0.6690, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0401, 0.0000, 0.9599]],
       grad_fn=<IndexBackward0>)
_nonzero_gates= tensor([[0.3310],
        [0.0401],
        [0.6690],
        [0.9599]], grad_fn=<GatherBackward0>)


In [27]:
inp=x
print("inp=",inp)
inp_exp = inp[_batch_index]
print("inp_exp=",inp_exp)
retu=torch.split(inp_exp, _part_sizes, dim=0)
print("return=",retu)

inp= tensor([[-0.1236, -0.1572,  0.0284, -1.8244],
        [ 0.2689, -0.0553, -2.3533, -0.2592]])
inp_exp= tensor([[-0.1236, -0.1572,  0.0284, -1.8244],
        [ 0.2689, -0.0553, -2.3533, -0.2592],
        [-0.1236, -0.1572,  0.0284, -1.8244],
        [ 0.2689, -0.0553, -2.3533, -0.2592]])
return= (tensor([[-0.1236, -0.1572,  0.0284, -1.8244]]), tensor([], size=(0, 4)), tensor([], size=(0, 4)), tensor([[ 0.2689, -0.0553, -2.3533, -0.2592]]), tensor([[-0.1236, -0.1572,  0.0284, -1.8244]]), tensor([[ 0.2689, -0.0553, -2.3533, -0.2592]]))


In [29]:
print(torch.split(inp_exp, [1,0,0,2,1,0],dim=0))

(tensor([[-0.1236, -0.1572,  0.0284, -1.8244]]), tensor([], size=(0, 4)), tensor([], size=(0, 4)), tensor([[ 0.2689, -0.0553, -2.3533, -0.2592],
        [-0.1236, -0.1572,  0.0284, -1.8244]]), tensor([[ 0.2689, -0.0553, -2.3533, -0.2592]]), tensor([], size=(0, 4)))


In [33]:
a=torch.tensor([[1,2],[0,3]])
a[a==0]=1
print(a)

tensor([[1, 2],
        [1, 3]])
